Estudiantes:
> Arlex Fabian Galindez Rivera

>  Jhony Mateo Rosero





Instalar la librerías

In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.5 MB/s eta 0:00:00


Importación de librerias

In [ ]:
import operator
import math
import random
import numpy
import itertools
import json
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

wityDefinir el conjunto
de funciones f que se van a usar, para generar
la expresión matemática o lógica que trata de
satisfacer la muestra finita de datos.

In [ ]:
pset = gp.PrimitiveSet("MAIN",6)
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_,1)
pset.addTerminal(1)
pset.addTerminal(0)
pset.renameArguments(ARG0="p")
pset.renameArguments(ARG1="q")
pset.renameArguments(ARG2="r")
pset.renameArguments(ARG3="s")
pset.renameArguments(ARG4="t")
pset.renameArguments(ARG4="u")

Definir el conjunto de datos de prueba, para este  caso se define una tabla de verdad, muestra cómo responde la salida de un circuito lógico a las varias combinaciones en las entradas

In [ ]:
samples=numpy.array([[0,0,0,0,0,0,0],
[0,0,0,0,0,1,0],
[0,0,0,0,1,0,0],
[0,0,0,0,1,1,0],
[0,0,0,1,0,0,0],
[0,0,0,1,0,1,0],
[0,0,0,1,1,0,0],
[0,0,0,1,1,1,0],
[0,0,1,0,0,0,1],
[0,0,1,0,0,1,1],
[0,0,1,0,1,0,1],
[0,0,1,0,1,1,1],
[0,0,1,1,0,0,1],
[0,0,1,1,0,1,1],
[0,0,1,1,1,0,1],
[0,0,1,1,1,1,1],
[0,1,0,0,0,0,0],
[0,1,0,0,0,1,0],
[0,1,0,0,1,0,0],
[0,1,0,0,1,1,0],
[0,1,0,1,0,0,1],
[0,1,0,1,0,1,1],
[0,1,0,1,1,0,1],
[0,1,0,1,1,1,1],
[0,1,1,0,0,0,0],
[0,1,1,0,0,1,0],
[0,1,1,0,1,0,0],
[0,1,1,0,1,1,0],
[0,1,1,1,0,0,1],
[0,1,1,1,0,1,1],
[0,1,1,1,1,0,1],
[0,1,1,1,1,1,1],
[1,0,0,0,0,0,0],
[1,0,0,0,0,1,0],
[1,0,0,0,1,0,1],
[1,0,0,0,1,1,1],
[1,0,0,1,0,0,0],
[1,0,0,1,0,1,0],
[1,0,0,1,1,0,1],
[1,0,0,1,1,1,1],
[1,0,1,0,0,0,0],
[1,0,1,0,0,1,0],
[1,0,1,0,1,0,1],
[1,0,1,0,1,1,1],
[1,0,1,1,0,0,0],
[1,0,1,1,0,1,0],
[1,0,1,1,1,0,1],
[1,0,1,1,1,1,1],
[1,1,0,0,0,0,0],
[1,1,0,0,0,1,1],
[1,1,0,0,1,0,0],
[1,1,0,0,1,1,1],
[1,1,0,1,0,0,0],
[1,1,0,1,0,1,1],
[1,1,0,1,1,0,0],
[1,1,0,1,1,1,1],
[1,1,1,0,0,0,0],
[1,1,1,0,0,1,1],
[1,1,1,0,1,0,0],
[1,1,1,0,1,1,1],
[1,1,1,1,0,0,0],
[1,1,1,1,0,1,1],
[1,1,1,1,1,0,0],
[1,1,1,1,1,1,1]])

Se define la función de evaluación en este caso se utiliza error cuadrático medio
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.3.1463&rep=rep1&type=pdf

In [ ]:
def evalSymbReg(individual):
    func = toolbox.compile(expr=individual)
    eval=math.sqrt((numpy.sum([math.pow((func(*row[:6])-row[6]),2) for row in samples]))/len(samples))
    #eval=numpy.sum([abs((func(*row[:6])-row[6])) for row in samples])
    return eval,


Definir hiperpárametros

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=6)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

Costruir el Algoritmo evolutivo (PG) y ejecutar

In [ ]:
def main():
    #random.seed(318)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.3, 100, stats, halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
    p,s,h=main()
    sol=h[0]
    print('mejor Solución encontrada',sol)
    print('longitud',len(sol))
    func = toolbox.compile(expr=sol)

    test=[func(*row[:6]) for row in samples]
    #print(samples)
    print(test)



gen	nevals	avg    	std      	min     	max     
0  	50    	0.67701	0.0539193	0.612372	0.790569
1  	31    	0.649181	0.0480981	0.612372	0.790569
2  	29    	0.630817	0.0454901	0.5     	0.790569
3  	33    	0.644347	0.0570522	0.5     	0.810093
4  	37    	0.636349	0.0642107	0.5     	0.800391
5  	36    	0.606051	0.0455759	0.5     	0.707107
6  	39    	0.608355	0.0521188	0.5     	0.790569
7  	38    	0.593442	0.0560232	0.5     	0.707107
8  	36    	0.55289 	0.0425739	0.5     	0.661438
9  	32    	0.529911	0.0371751	0.5     	0.637377
10 	32    	0.53193 	0.0574478	0.5     	0.707107
11 	26    	0.513604	0.0390318	0.5     	0.684653
12 	29    	0.538221	0.0685077	0.5     	0.75    
13 	37    	0.52177 	0.0524983	0.5     	0.790569
14 	31    	0.522864	0.0504998	0.5     	0.707107
15 	28    	0.524238	0.05465  	0.5     	0.707107
16 	32    	0.511336	0.0367119	0.414578	0.637377
17 	33    	0.523094	0.0541806	0.5     	0.707107
18 	38    	0.5246  	0.0478994	0.5     	0.707107
19 	34    	0.52523 	0.0475268	0.5     	0.7